In [107]:
import torch
import torch.nn as nn
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, GPT2Model, GPT2LMHeadModel
from transformers.generation.configuration_utils import GenerationConfig

In [108]:
# params
n_embed = 768
batch_size = 16

# RL training params
n_episodes = 100
n_landmarks = 3
device = "cuda" if torch.cuda.is_available() else "cpu"

# IO dimensions
speaker_in = 3
speaker_out = 5
listener_in = n_landmarks*2
listener_out = 1

# speaker generation params
num_beams = 5
early_stopping = True
no_repeat_ngram_size = 1
do_sample = True
temperature = 0.9
top_k = 50
top_p = 0.9
max_length = speaker_out


# landamrk colors
landmarks_c = sns.color_palette(n_colors=n_landmarks)
landmarks_c = torch.tensor(landmarks_c)


In [109]:
class Speaker(nn.Module):
  def __init__(self) -> None:
    super().__init__()

    # init GPT2
    self.tokenizer = AutoTokenizer.from_pretrained("gpt2")
    self.transformer = GPT2LMHeadModel.from_pretrained("gpt2")

    # config file when generating text
    self.generation_config = GenerationConfig.from_model_config(self.transformer.config)

    # freeze GPT2 params
    for param in self.transformer.parameters():
      param.requires_grad = False

    # custom embedding layer for obs
    self.embed_obs = nn.Linear(speaker_in, n_embed)


  def forward(self, obs: torch.Tensor):
    ''' 
    The observation of the speaker is the color of the target landmark of the listener. 
    Given the color, the speaker returns a message to the listener.
    '''

    # dims
    batch_size = obs.shape[0]

    # compute embeddings of observation (the color)
    obs_embeddings = self.embed_obs(obs)
    attention_mask = torch.ones((batch_size, obs_embeddings.shape[1]))
    # generate text (sampling based)
    msg = self.transformer.generate(
                          inputs_embeds=obs_embeddings,
                          attention_mask=attention_mask, 
                          generation_config=self.generation_config, 
                          num_beams=num_beams,
                          early_stopping=early_stopping,
                          no_repeat_ngram_size=no_repeat_ngram_size,
                          do_sample=do_sample,
                          temperature=temperature,
                          top_k=top_k,
                          top_p=top_p,
                          max_length=max_length,
                          pad_token_id=self.tokenizer.eos_token_id)

    return msg


class Listener(nn.Module):
  def __init__(self) -> None:
    super().__init__()

    # init GPT2
    self.tokenizer = AutoTokenizer.from_pretrained("gpt2")
    self.transformer = GPT2Model.from_pretrained("gpt2")

    # freeze GPT2 params
    for param in self.transformer.parameters():
      param.requires_grad = False

    # GPT2 word embedding layer
    self.wte = nn.Embedding.from_pretrained(self.transformer.wte.weight)

    # custom embedding layer for processing input obs
    self.embed_obs = nn.Linear(listener_in, n_embed)

    # custom prediction layer for computing the action
    self.predict_action = nn.Linear(n_embed*(1+speaker_out), listener_out)

  def forward(self, obs:torch.Tensor, msg:torch.Tensor):
    '''
    obs: position of landmarks
    msg: message from speaker indicating which landmark to go to
    '''

    # compute and stack embeddings
    word_embeddings = self.wte(msg)
    obs_embeddings = self.embed_obs(obs).unsqueeze(1)
    stacked_inputs = torch.cat((word_embeddings, obs_embeddings), 1)

    # compute transformer hidden states
    hidden_state = self.transformer(inputs_embeds=stacked_inputs)['last_hidden_state']

    # reshape hidden states and pass through prediction layer
    x = hidden_state.view((batch_size, -1))
    action = self.predict_action(x)

    return action

In [ ]:
# init models
speaker = Speaker().to(device)
listener = Listener().to(device)

# init optimizer
optimizer = torch.optim.Adam(params=list(speaker.parameters())+list(listener.parameters()), lr=1e-3)

# loss function
lossfun = nn.L1Loss()

In [ ]:
loss_history = []

for i in range(n_episodes):
  # random landmark position
  landmarks_p = (torch.rand((batch_size, 2*n_landmarks)) - 0.5) * 2
  # sample target landmark indices
  ids = torch.randint(n_landmarks, (batch_size,))
  # speaker input
  goal_landmarks = landmarks_c.repeat(batch_size, 1)[ids]
  # messages from speaker
  msg = speaker(goal_landmarks)
  # compute action from lisener
  action = listener(landmarks_p, msg)
  # ground truth target actions
  target = torch.cat([landmarks_p[i, ix*2:(ix+1)*2].unsqueeze(0) for i, ix in enumerate(ids)], 0)
  # backprop
  optimizer.zero_grad()
  loss = lossfun(action, target)
  loss.backward()
  optimizer.step()
  
  if i%10==0:
    print(f'At {int(i/n_episodes * 100)}%')
    loss_history.append(loss.item())

plt.plot(loss_history)

In [ ]:
ids = torch.randint(n_landmarks, (batch_size,))

print(ids)

print(landmarks_c)
print(landmarks_c.repeat(batch_size, 1)[ids])

print(landmarks_p)
#print(landmarks_p[:, [0,1]])

print(target)


In [29]:
speaker = Speaker()

obs_speaker = torch.rand((batch_size, 1, speaker_in))

msg = speaker(obs_speaker)

In [56]:
listener = Listener()

obs_listener = torch.rand((batch_size, listener_in))

action = listener(obs_listener, msg)


In [57]:
print(action)

tensor([[ -6.2932],
        [ -5.7160],
        [ -6.1091],
        [ -6.3199],
        [ -7.1832],
        [ -6.8462],
        [ -4.7256],
        [ -4.2928],
        [ -6.8898],
        [-12.8584],
        [ -7.2163],
        [ -4.9117],
        [ -5.0960],
        [ -8.3092],
        [-11.7970],
        [-12.8817]], grad_fn=<AddmmBackward0>)


In [53]:
n_embed*(1+speaker_out)

4608

In [30]:
for m in msg:
  print(speaker.tokenizer.decode(m, skip_special_tokens=True))

print(msg.shape)

 is it?

 of the people who
 to get the best
 is that?

.


The
 I have never been
 the fact that he
 to keep up with
. I mean,
. I don't
.
So,
 to make sure that
 of the United States
 I don't know
, you don't
. I don't
torch.Size([16, 5])
